# 머신러닝
* TensorFlow를 활용한 머신러닝

## 텐서플로우의 이해
* Tensor란 : 다차원(3차원 이상) 배열로 나타내는 데이터
    
* Flow란 : 흐름 데이터 플로우 그래프를 만드는 것

* 노드에 있는 데이터들이 엣지를 따라 흘러서 계산을 한다(?)

### 텐서플로우 실행하기

In [1]:
import tensorflow as tf

In [2]:
hello = tf.constant('Hello TensorFlow!') #텐서플로우 상수 만들기 .constant
print(hello) #결과가 Tesor(~)로 나옴

sess = tf.Session()
result = sess.run(hello)
print(result) #b - byte string 기계어라고 보면 됨
sess.close()

Tensor("Const:0", shape=(), dtype=string)
b'Hello TensorFlow!'


### 계산그래프 만들기

* 빌딩구조와 실행구조의 분리
    * dataflow graph : 그래프의 정의 - 어떻게 동작 할 것이다라는 계획
    * session : 실행구조 - 이렇게 해라 라는 실행

In [3]:
node1 = tf.constant(3, tf.float32) #두개가 type이 다르면 계산이 안됨, 그래서 보통 실수로 맞춘다.
node2 = tf.constant(4.0)
node3 = tf.add(node1, node2) #세번째 노드를 만들 때 엣지가 같이 생성됨

In [4]:
print("node1: ", node1)
print("node2: ", node2)
print("node3: ", node3)

node1:  Tensor("Const_1:0", shape=(), dtype=float32)
node2:  Tensor("Const_2:0", shape=(), dtype=float32)
node3:  Tensor("Add:0", shape=(), dtype=float32)


In [5]:
session = tf.Session()
print("session.run([node1,node2]): ", session.run([node1, node2]))
print("session.run(node3): ", session.run(node3))

session.close() 

#Session이 run이 되어야 그래프에 데이터가 흐르면서 계산이 됨
# Tensor가 Flow한다.

session.run([node1,node2]):  [3.0, 4.0]
session.run(node3):  7.0


* 구조의 장단점
    * 장점 
        * gradient 자동 계산(Back Propagation)하기 쉬움
        * 프로세스 병렬 구성 쉬움 - 동시 실행 가능
        * 동일 모델을 다양한 장치에서 사용 쉬움(★분산 처리) - 시간절약
        * 내부 구조 시각화 - 그래프로 확인 가능
    
    * 단점
        * 디버깅이 어려움

### 텐서플로우의 자료형

In [6]:
#다른 type으로 - 정수를 실수로 ; shape가 없으니 0차원-스칼라
a = tf.constant(1, dtype=tf.float32)
print(a)
print(a.shape)
with tf.Session() as session :
    print(session.run(a))

Tensor("Const_3:0", shape=(), dtype=float32)
()
1.0


In [7]:
#다른 shape로 - 몇 차원(행렬)인지
a = tf.constant(1, dtype = tf.float32, shape=(1,))
print(a)
print(a.shape)
with tf.Session() as session :
    print(session.run(a))

Tensor("Const_4:0", shape=(1,), dtype=float32)
(1,)
[1.]


In [8]:
#2차원으로 설정
a = tf.constant(1, dtype = tf.float32, shape=(1,1))
print(a)
print(a.shape)
with tf.Session() as session :
    print(session.run(a))

Tensor("Const_5:0", shape=(1, 1), dtype=float32)
(1, 1)
[[1.]]


### placeholder
    * 특징: 갈아끼우기 가능 - x(feature)값에 사용

In [9]:
input_data = [1,2,3]
x = tf.placeholder(dtype=tf.float32) #값을 실행단계에서 넣어준다!

y = x*2
print("x: ",x)
print("y: ",y)

session = tf.Session()

#feed_dict를 통해 값을 전달
result = session.run(y, feed_dict={x:input_data})
print(result)
session.close()

#결과에서 0 생략가능 2.0 -> 2.

x:  Tensor("Placeholder:0", dtype=float32)
y:  Tensor("mul:0", dtype=float32)
[2. 4. 6.]


In [10]:
x = tf.placeholder(dtype=tf.float32)
x_data = [1,2,4]

#초기값: tf.random_normal([1,3], stddev=0.35)
weights = tf.Variable([3], dtype = tf.float32)
biases = tf.Variable([4], dtype = tf.float32)

y = weights*x + biases

print("x: ", x)
print("weights: ", weights)
print("biases: ", biases)

x:  Tensor("Placeholder_1:0", dtype=float32)
weights:  <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>
biases:  <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


### 개별 initializer

In [11]:
with tf.Session() as session:
    #초기화 오퍼레이션을 실행 - 없으면 오류(초기값을 할당한 후 initializer해줘야함)
    session.run([weights.initializer, biases.initializer]) # list로 run하면 실행을 한번에함
    
    result = session.run(y, feed_dict={x:x_data})
    print(result)
    
    #변수 변경 // 변수 재 할당
    session.run(tf.assign(weights, [2]))
    
    #다시 실행
    result = session.run(y, feed_dict={x:x_data})
    print(result)

[ 7. 10. 16.]
[ 6.  8. 12.]


### 전체 변수 한번에 초기화: global_variables_initializer()
    * 변수가 많아져도 한번에 가능

In [12]:
#변수 초기화 작업
init_op = tf.global_variables_initializer()

with tf.Session() as session:
    #초기화 오퍼레이션을 실행 
    session.run(init_op)
    
    result = session.run(y, feed_dict={x:x_data})
    print(result)
    
    #변수 변경 // 변수 재 할당
    session.run(tf.assign(weights, [2]))
    
    #다시 실행
    result = session.run(y, feed_dict={x:x_data})
    print(result)

[ 7. 10. 16.]
[ 6.  8. 12.]


## 텐서의 속성 (Rank, Shape, Types)

In [13]:
# import tensorflow as tf

a = tf.constant(1) # 0차원 = scalar
b = tf.constant([1,2], dtype =tf.float32) # 1차원 = vector
c = tf.constant([[1,2],[3,4]], dtype = tf.int32) # 2차원, = matrix
d = tf.constant(
    [
        [[1.0,2.0], [3,4]],
            [[5,6],[7, 8]]
    ]
) # 3차원 =3-Tensor

print("a: ", a)
print("b: ", b)
print("c: ", c)
print("d: ", d)

a:  Tensor("Const_6:0", shape=(), dtype=int32)
b:  Tensor("Const_7:0", shape=(2,), dtype=float32)
c:  Tensor("Const_8:0", shape=(2, 2), dtype=int32)
d:  Tensor("Const_9:0", shape=(2, 2, 2), dtype=float32)


### Rank

In [14]:
print("a's rank: ", a._rank())
print("b's rank: ", b._rank())
print("c's rank: ", c._rank())
print("d's rank: ", d._rank())

print("-"*50)
# 아래는 텐서로 반환
print("a's rank: ", tf.rank(a))
print("b's rank: ", tf.rank(b))
print("c's rank: ", tf.rank(c))
print("d's rank: ", tf.rank(d))

a's rank:  0
b's rank:  1
c's rank:  2
d's rank:  3
--------------------------------------------------
a's rank:  Tensor("Rank:0", shape=(), dtype=int32)
b's rank:  Tensor("Rank_1:0", shape=(), dtype=int32)
c's rank:  Tensor("Rank_2:0", shape=(), dtype=int32)
d's rank:  Tensor("Rank_3:0", shape=(), dtype=int32)


### Shape

In [15]:
print("a's shape: ", a.get_shape())
print("b's shape: ", b.get_shape())
print("c's shape: ", c.get_shape())
print("d's shape: ", d.get_shape())

print("-"*50)
# 아래는 텐서로 반환
print("a's shape: ", tf.shape(a))
print("b's shape: ", tf.shape(b))
print("c's shape: ", tf.shape(c))
print("d's shape: ", tf.shape(d))

a's shape:  ()
b's shape:  (2,)
c's shape:  (2, 2)
d's shape:  (2, 2, 2)
--------------------------------------------------
a's shape:  Tensor("Shape:0", shape=(0,), dtype=int32)
b's shape:  Tensor("Shape_1:0", shape=(1,), dtype=int32)
c's shape:  Tensor("Shape_2:0", shape=(2,), dtype=int32)
d's shape:  Tensor("Shape_3:0", shape=(3,), dtype=int32)


### dtype

In [16]:
print("a's dtype: ", a.dtype)
print("b's dtype: ", b.dtype)
print("c's dtype: ", c.dtype)
print("d's dtype: ", d.dtype) # 섞여있으면 범위가 큰 쪽으로 정수 < 실수

a's dtype:  <dtype: 'int32'>
b's dtype:  <dtype: 'float32'>
c's dtype:  <dtype: 'int32'>
d's dtype:  <dtype: 'float32'>


* Tensorflow의 DataType은 크게 float, complex, int, unsigned int, bool, string, 등

## Session관리

### tf.get_default_session()

In [17]:
session = tf.get_default_session() # 함수 - Session 객체를 만들지 않아서 None이 뜸
print(session)

None


In [18]:
with tf.Session() as sess :
    print(tf.get_default_session())

### eval(),initializer.run()

In [20]:
a = tf.constant(2, dtype = tf.float32)
b = tf.Variable(3, dtype = tf.float32)
c = a * b

In [21]:
with tf.Session() as sess:
    print(sess.run(b.initializer))
    print(sess.run(a))
    print(sess.run(c))
    
    print('-'*50)
    # 똑같은 결과 
    print(b.initializer.run())
    # tf.get_default_session().run(b.initializer)
    print(a.eval())
    print(c.eval())

None
2.0
6.0
--------------------------------------------------
None
2.0
6.0


* 익숙한 with절을 사용, session이 자동으로 닫힘

### tf.InteractiveSession

In [22]:
sess = tf.InteractiveSession()
sess.run(b.initializer)
print(a.eval())
print(c.eval())
sess.close()

2.0
6.0


* session을 닫아줘야함, run과 eval의 사용; 디버깅이 쉬움

## Graph 관리

### get_default_graph()

In [23]:
tf.get_default_graph() # 초기 그래프 객체 불러옴

In [24]:
a= tf.Variable(3)
a.graph is tf.get_default_graph() 
"""
텐서를 생성하면 텐서가 초기 그래프에 그려져서 
텐서의 그래프가 초기 그래프냐고 물으면 True,
그래프 객체로 초기 그래프가 아닌 그래프를 생성할 수 있음
"""

True

### Graph객체

* 초기 그래프가 아닌 그래프를 그리는 이유 :
    * 서로 다른 모델링을 할 때 각 그래프에 생성하여 비교할 수 있음
    * 예를 들어 default그래프에는 단순선형모델, graph 그래프에는 3차함수모델(곡선)

In [25]:
graph = tf.Graph()
with graph.as_default():
    b = tf.Variable(2)

In [27]:
b.graph is graph #초기 그래프가 아닌 graph로 정의된 그래프에 그려진다.

True

In [28]:
tf.get_default_graph().get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'Const_2' type=Const>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'Const_3' type=Const>,
 <tf.Operation 'Const_4' type=Const>,
 <tf.Operation 'Const_5' type=Const>,
 <tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'mul/y' type=Const>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'Variable_1/initial_value' type=Const>,
 <tf.Operation 'Variable_1' type=VariableV2>,
 <tf.Operation 'Variable_1/Assign' type=Assign>,
 <tf.Operation 'Variable_1/read' type=Identity>,
 <tf.Operation 'mul_1' type=Mul>,
 <tf.Operation 'add_1' type=Add>,
 <tf.Operation 'Assign/value' type=Const>,
 <tf.Operation 'Assign' type=Assign>,
 <tf.Operation 'init' type=NoOp>,
 <t

In [29]:
tf.reset_default_graph() # 그래프 초기화
tf.get_default_graph().get_operations() # 초기그래프의 노드들 확인

[]